## Offshore Wind Farming

with integer programming and Gurobi

Source: http://examples.gurobi.com/offshore-wind-farming/#demo

通用问题 <a href=https://books.google.co.uk/books?id=Idd9CAAAQBAJ> fixed charge network flow </a> 通信网络或运输网络的规划问题；解决水下电缆的安置问题；从风车收集电力

In this example we'll solve the problem of how to minimize the cost of laying underwater cables to collect electricity produced by an offshore wind farm.

构建数学模型，并可视化方案

We'll construct a mathematical model of the business problem, implement this model in Gurobi's Python interface, and compute and visualize an optimal solution.


Although your own business may not involve operating a wind farm, the same basic techniques used in this example can be used for other applications like the planning of communication and transportation networks.

<img height=400 width=600 src=http://examples.gurobi.com/offshore-wind-farming/screenshot.png>

* 白圈 - 风车 (必选生产节点） Turbines are represented by white circles: 
* 黑圈 - 转换站 （可选节点） Transfer stations are represented by black circles: 
* 绿圈 - 电站 （必选消耗节点）The power station is represented by a green circle: 

如果转换站的路由不是很理想，并且敷设电缆的固定费用较低（<=3500\$）的话，有些转换站将不被使用。

如果转换站的路由很好（围绕一圈都是电站），并且敷设电缆的固定费用较高（>=2500\$）的话，该转换站被使用。

## Problem Description

<img height=200 width=300 src=http://examples.gurobi.com/offshore-wind-farming/windfarm.jpg>

### Implementation

In [1]:
from gurobipy import *

# Vertices and associated supply
vertices = {0: 4, 1: 3, 2: 2, 3: 0, 4: -6, 5: -3}

# Dict key: edge, value: (capacity, cost per flow, fixed cost)
edges = {(0,4): (4,1,1),
         (0,3): (2,1,1),
         (1,3): (3,1,1),
         (2,5): (2,1,1),
         (3,4): (2,1,1),
         (3,5): (1,1,1)}

m = Model()

x = {} # Flow on each edge
y = {} # Binary variable for each edge

# key: vertex, value: list of edges entering/leaving the vertex
edgeIn   = { v:[] for v in vertices }
edgeOut  = { v:[] for v in vertices }

# Add variables
for edge in edges:
  u = edge[0]
  v = edge[1]
  y[edge] = m.addVar(vtype=GRB.BINARY, name="y" + str(edge))
  x[edge] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x" + str(edge) )
  edgeIn[v] = edgeIn[v] + [x[edge]]
  edgeOut[u] = edgeOut[u] + [x[edge]]

m.update()

# Add constraints
for v in vertices:
  m.addConstr(quicksum(edgeOut[v]) - quicksum(edgeIn[v]) == vertices[v], name="v%d" % v)

for edge in edges:
  m.addConstr(x[edge] <= edges[edge][0]*y[edge], name=str(edge))

# Set objective
m.setObjective(quicksum((edges[edge][1]*x[edge] + edges[edge][2]*y[edge]) for edge in edges))
m.optimize()

Optimize a model with 12 rows, 12 columns and 24 nonzeros
Variable types: 6 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 6e+00]
Presolve removed 12 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 17 
Pool objective bound 17

Optimal solution found (tolerance 1.00e-04)
Best objective 1.700000000000e+01, best bound 1.700000000000e+01, gap 0.0000%


## Optimal Result


In [2]:
for v in m.getVars():
    #if v.X != 0:
        print("%s %f" % (v.Varname, v.X))

y(1, 3) 1.000000
x(1, 3) 3.000000
y(0, 4) 1.000000
x(0, 4) 4.000000
y(0, 3) 0.000000
x(0, 3) 0.000000
y(3, 4) 1.000000
x(3, 4) 2.000000
y(2, 5) 1.000000
x(2, 5) 2.000000
y(3, 5) 1.000000
x(3, 5) 1.000000
